In [13]:
import random

import spacy
from spacy.training.example import Example


nlp = spacy.blank("en")

## Prepare Training Data

First, we have to know the exact position of the entities in a string

In [ ]:
text_1 = "GREEN FIELD\n5305 E PACIFIC COAST HWY\nLong Beach, CA 90804\n(562) 597-0906\n\nServer: Francis         Station: 3\n----------------------------------\nOrder #: 69923          Dine In\nTable: B11              Guests: 2\n----------------------------------\n1 Coffee                3.00\n2 Lunch                 45.90\n1 Coke                  3.00\n\nSUB TOTAL:              51.90\nTax 1:                  4.60\n\nTOTAL:                  $56.58\n\n    5/26/2016 12:53:10 PM\n\n        THANK YOU!"

entities_1 = [
    '1 Coffee                3.00',
    '2 Lunch                 45.90',
    '1 Coke                  3.00'
]

for entity in entities_1:
    starting_index = text_1.index(entity)
    print(f'Entity: "{entity}" starts at: {starting_index}, ends at: {starting_index + len(entity)}')

In [1]:
text_1 = "zZ, Die Gastronorni eirußppe\n\nHochschule Luzern\nInformatik & Wirtschaft\nc/o ZFV-Unternehmungen\nFlüelastrasse 51\n8047 Zürich\n\nCHF\n\nPizza Neapolitana 10.50\nRamseier äpfelschor) 3.00\n\nTotal CHF 13,50\n\nZ MASTERCARD 13.50\n\nMwSt\n\nMwSt Prozent Netto Brutto\n(1) 8.10 12.49 13.50 1.01\n\nProfit Center: InHouse\n23.11.2024 11:36:49 #:2450 0p:568071 Cı56\n8002 $:568\n\nEs bediente Sie - selflheck\n\nKopie # |\n\nCHE-105..827.102 MWST\nVielen Dank und auf Wiedersehen\n\n"

entities_1 = [
    'Pizza Neapolitana 10.50',
    'Ramseier äpfelschor) 3.00'
]

for entity in entities_1:
    starting_index = text_1.index(entity)
    print(f'Entity: "{entity}" starts at: {starting_index}, ends at: {starting_index + len(entity)}')

Entity: "Pizza Neapolitana 10.50" starts at: 130, ends at: 153
Entity: "Ramseier äpfelschor) 3.00" starts at: 154, ends at: 179
